# Generate all-to-all proximities using random walk with restart

In [ ]:
import numpy as np
import scipy.sparse
import sknetwork as skn

In [ ]:
matrix = scipy.sparse.load_npz("data/score-matrices/combined_score.sparse.npz")
matrix = matrix / 1000  # rescale STRING scores to be from 0 to 1

In [ ]:
def get_rwr_scores(seeds):
    pagerank = skn.ranking.PageRank(damping_factor=0.85)
    np.random.seed(0)  # set seed in case there is some non-determinism
    scores = pagerank.fit_transform(adjacency=matrix, seeds=seeds)
    scores /= scores.mean()  # rescale scores to have a mean of 1
    return scores

In [ ]:
nrows = matrix.shape[0]
nrows

In [ ]:
null_scores = get_rwr_scores(seeds=np.ones(nrows))
null_scores

In [ ]:
%%time
rwr_matrix = np.zeros(matrix.shape)
for i in range(nrows):
    scores = get_rwr_scores(seeds={i: 1.0})
    scores -= null_scores  # adjust scores for null scores
    rwr_matrix[i] = scores  # set values in row i to contain scores
rwr_matrix

In [ ]:
# density of score matrix
np.count_nonzero(rwr_matrix) / rwr_matrix.size